# ESERCITAZIONE 2 Analytics

innanzitutto non abbiamo un dataset unico da splittare bensi un train su cui allenare il nostro algoritmo e il test su cui fittare l'algoritmo e calcolare alcune semplice metriche:
- accuracy
- confusion matrix

In [55]:
#importiamo il train mediante la libreria di alto livello Pandas
import pandas as pd
train=pd.read_csv('/Users/alfonsodamelio/Downloads/creditcardTRAIN.csv',delimiter=',')
test=pd.read_csv('/Users/alfonsodamelio/Downloads/creditcardTEST.csv',delimiter=',')
#metto una variabile che dopo mi aiuti a riconoscere il train ed il test
train['label'] = 'train'
test['label'] = 'test'
# Concat
concat_df = pd.concat([train , test])

In [56]:
print('train length: %d'%len(train))
print('test length: %d'%len(test))

train length: 20000
test length: 10000


In [57]:
concat_df.head(5)

,AGE,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,DEFAULT,EDUCATION,ID,...,PAY_5,PAY_6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,SEX,label
0,24,3913,3102,689,0,0,0,1.0,2,1,...,-2,-2,0,689,0,0,0,0,2,train
1,26,2682,1725,2682,3272,3455,3261,1.0,2,2,...,0,2,0,1000,1000,1000,0,2000,2,train
2,34,29239,14027,13559,14331,14948,15549,0.0,2,3,...,0,0,1518,1500,1000,1000,1000,5000,2,train
3,37,46990,48233,49291,28314,28959,29547,0.0,2,4,...,0,0,2000,2019,1200,1100,1069,1000,2,train
4,57,8617,5670,35835,20940,19146,19131,0.0,2,5,...,0,0,2000,36681,10000,9000,689,679,1,train


In [58]:
#la prima cosa che facciamo è andare a togliere la feature ID, in quanto inutile ai fine della classificazione
concat_df=concat_df.drop(['ID'],axis=1)

In [59]:
#nuovamente display del train
concat_df.head(5)
#possiamo vedere che ora la variabile ID non c'è più.

,AGE,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,DEFAULT,EDUCATION,LIMIT_BAL,...,PAY_5,PAY_6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,SEX,label
0,24,3913,3102,689,0,0,0,1.0,2,20000,...,-2,-2,0,689,0,0,0,0,2,train
1,26,2682,1725,2682,3272,3455,3261,1.0,2,120000,...,0,2,0,1000,1000,1000,0,2000,2,train
2,34,29239,14027,13559,14331,14948,15549,0.0,2,90000,...,0,0,1518,1500,1000,1000,1000,5000,2,train
3,37,46990,48233,49291,28314,28959,29547,0.0,2,50000,...,0,0,2000,2019,1200,1100,1069,1000,2,train
4,57,8617,5670,35835,20940,19146,19131,0.0,2,50000,...,0,0,2000,36681,10000,9000,689,679,1,train


In [60]:
features_df = pd.get_dummies(concat_df, columns=['SEX','EDUCATION','MARRIAGE','PAY_0','PAY_2','PAY_3','PAY_4','PAY_5','PAY_6'],dummy_na=True)

In [61]:
features_df.head(5)

,AGE,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,DEFAULT,LIMIT_BAL,PAY_AMT1,...,PAY_6_-1.0,PAY_6_0.0,PAY_6_2.0,PAY_6_3.0,PAY_6_4.0,PAY_6_5.0,PAY_6_6.0,PAY_6_7.0,PAY_6_8.0,PAY_6_nan
0,24,3913,3102,689,0,0,0,1.0,20000,0,...,0,0,0,0,0,0,0,0,0,0
1,26,2682,1725,2682,3272,3455,3261,1.0,120000,0,...,0,0,1,0,0,0,0,0,0,0
2,34,29239,14027,13559,14331,14948,15549,0.0,90000,1518,...,0,1,0,0,0,0,0,0,0,0
3,37,46990,48233,49291,28314,28959,29547,0.0,50000,2000,...,0,1,0,0,0,0,0,0,0,0
4,57,8617,5670,35835,20940,19146,19131,0.0,50000,2000,...,0,1,0,0,0,0,0,0,0,0


In [62]:
# Split your data
train = features_df[features_df['label'] == 'train']
test = features_df[features_df['label'] == 'test']

# Drop your labels
train = train.drop('label', axis=1)
test = test.drop('label', axis=1)

In [63]:
len(train.columns)

101

In [64]:
len(test.columns)

101

In [65]:
#creiamo la X e la y
X=train.drop(['DEFAULT'],axis=1).values
y=train['DEFAULT'].values

In [66]:
#per semplicita dividiamo in train e test anche il nostro train almeno siamo in grado di calcolare qualche metriche per vedere come il nostro algoritmo risponde o meglio "impara"
from sklearn.cross_validation import train_test_split, cross_val_score
from sklearn.feature_selection import SelectKBest, chi2, f_classif

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

In [72]:
from sklearn.ensemble import RandomForestClassifier
Rf=RandomForestClassifier()
Rf.fit(X_train, y_train)

# Predicting the Test set results
y_pred = Rf.predict(X_test)

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

[[4281  331]
 [ 906 3748]]


0.86650118713576518

In [47]:
from sklearn.ensemble import GradientBoostingClassifier
classifier = GradientBoostingClassifier()
#min_samples_leaf=15
classifier.fit(X_train, y_train)

# Predicting the Test set results
y_pred = classifier.predict(X_test)

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

[[4373  239]
 [ 916 3738]]


0.87535074465788909

In [73]:
test=test.drop('DEFAULT',axis=1).values

AttributeError: 'numpy.ndarray' object has no attribute 'drop'

In [74]:
y_stim=classifier.predict(test)

In [40]:
#importiamo il train mediante la libreria di alto livello Pandas
import pandas as pd
train=pd.read_csv('/Users/alfonsodamelio/Downloads/creditcardTRAIN.csv',delimiter=',')
test=pd.read_csv('/Users/alfonsodamelio/Downloads/creditcardTEST.csv',delimiter=',')
#metto una variabile che dopo mi aiuti a riconoscere il train ed il test
train['label'] = 'train'
test['label'] = 'test'
# Concat
concat_df = pd.concat([train , test])
concat_df = concat_df.drop('ID',axis=1)

In [41]:
concat_df

,AGE,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,DEFAULT,EDUCATION,LIMIT_BAL,...,PAY_5,PAY_6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,SEX,label
0,24,3913,3102,689,0,0,0,1.0,2,20000,...,-2,-2,0,689,0,0,0,0,2,train
1,26,2682,1725,2682,3272,3455,3261,1.0,2,120000,...,0,2,0,1000,1000,1000,0,2000,2,train
2,34,29239,14027,13559,14331,14948,15549,0.0,2,90000,...,0,0,1518,1500,1000,1000,1000,5000,2,train
3,37,46990,48233,49291,28314,28959,29547,0.0,2,50000,...,0,0,2000,2019,1200,1100,1069,1000,2,train
4,57,8617,5670,35835,20940,19146,19131,0.0,2,50000,...,0,0,2000,36681,10000,9000,689,679,1,train
5,37,64400,57069,57608,19394,19619,20024,0.0,1,50000,...,0,0,2500,1815,657,1000,1000,800,1,train
6,29,367965,412023,445007,542653,483003,473944,0.0,1,500000,...,0,0,55000,40000,38000,20239,13750,13770,1,train
7,23,11876,380,601,221,-159,567,0.0,2,100000,...,0,-1,380,601,0,581,1687,1542,2,train
8,28,11285,14096,12108,12211,11793,3719,0.0,3,140000,...,0,0,3329,0,432,1000,1000,1000,2,train
9,35,0,0,0,0,13007,13912,0.0,3,20000,...,-1,-1,0,0,0,13007,1122,0,1,train


A questo punto procediamo nel seguente modo:
* Splittare il train in $\rightarrow$ $X$ ed $y$ 
* pre processing delle variabili nella $X$
    - ovvero se abbiamo a che fare con variabili qualitative o categoriche utilizziamo "label encoder & one hot encoder" per renderle numeriche ed interpretare i vari livelli ... e magari standardadizzare le variabile
* Effettuata la pulizia del data set siamo pronti per allenare i nostri algoritmi di **Decision trees**
    - Random Forest
    - Gradient Boosting
    
Ovviamente per poi fare una predizione sul test, dobbiamo effettuare la stessa pulizia sul test, il quale ovviamente è privo della variabile target

In [42]:
#creiamo la X e la y
X=concat_df.drop(['DEFAULT'],axis=1).values
y=concat_df['DEFAULT'].values

In [46]:
y=y[:20000]

In [47]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
#transform the first column in numbers
qualitative = [23]
for c in qualitative:
    labelencoder_X = LabelEncoder()
    X[:, c] = labelencoder_X.fit_transform(X[:, c])

In [48]:
#numero di features
print('# features: %d'%len(X[1,:]))
#numero di righe
print('# rows: %d'%len(X))

# features: 24
# rows: 30000


In [49]:
X

array([[24, 3913, 3102, ..., 0, 2, 1],
       [26, 2682, 1725, ..., 2000, 2, 1],
       [34, 29239, 14027, ..., 5000, 2, 1],
       ..., 
       [37, 3565, 3356, ..., 3100, 1, 0],
       [41, -1645, 78379, ..., 1804, 1, 0],
       [46, 47929, 48905, ..., 1000, 1, 0]], dtype=object)

In [50]:
#importiamo le librerie di pre-processing che ci servono dalla libreria di alto livello SCIKITLEARN
import warnings #eliminiamo i warnings
warnings.filterwarnings("ignore")

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.cross_validation import train_test_split, cross_val_score
from sklearn.feature_selection import SelectKBest, chi2, f_classif

A questo punto siamo pronti per l'encoding delle variabili categoriche:
- SEX $\Rightarrow$ index=1
- EDUCATION $\Rightarrow$ index=2
- MARRIAGE $\Rightarrow$ index=3
- PAY_0-PAY_6 $\Rightarrow$ index=5,6,7,8,9,10



In [51]:
#non possiamo fare l'encoding di fattori negativi quindi dobbiamo convertire -1,-2 in 10 e 11
import numpy as np
#pay_0
pay0=[i for i in X[:,10]]
for n, i in enumerate(pay0):
    if i == -1:
        pay0[n] = 10
    if i == -2:
        pay0[n] = 11
X[:,10]=np.array(pay0)

#pay_2
pay2=[i for i in X[:,11]]
for n, i in enumerate(pay2):
    if i == -1:
        pay2[n] = 10
    if i == -2:
        pay2[n] = 11
X[:,11]=np.array(pay2)

#pay_3
pay3=[i for i in X[:,12]]
for n, i in enumerate(pay3):
    if i == -1:
        pay3[n] = 10
    if i == -2:
        pay3[n] = 11
X[:,12]=np.array(pay3)

#pay_4
pay4=[i for i in X[:,13]]
for n, i in enumerate(pay4):
    if i == -1:
        pay4[n] = 10
    if i == -2:
        pay4[n] = 11
X[:,13]=np.array(pay4)

#pay_5
pay5=[i for i in X[:,14]]
for n, i in enumerate(pay5):
    if i == -1:
        pay5[n] = 10
    if i == -2:
        pay5[n] = 11
X[:,14]=np.array(pay5)

#pay_6
pay6=[i for i in X[:,15]]
for n, i in enumerate(pay6):
    if i == -1:
        pay6[n] = 10
    if i == -2:
        pay6[n] = 11
X[:,15]=np.array(pay6)


In [52]:
#facciamo l'encoding delle variabili sopra elencate utilizzando i rispettivi indici di lato rappresentati
onehotencoder = OneHotEncoder(categorical_features = [7,9,10,11,12,13,14,15,22])
X = onehotencoder.fit_transform(X).toarray()

In [53]:
#facciamo lo stesso encoding per la variabile target
labelencoder_y = LabelEncoder()
y = labelencoder_y.fit_transform(y)

In [54]:
X

array([[  0.00000000e+00,   0.00000000e+00,   1.00000000e+00, ...,
          0.00000000e+00,   0.00000000e+00,   1.00000000e+00],
       [  0.00000000e+00,   0.00000000e+00,   1.00000000e+00, ...,
          0.00000000e+00,   2.00000000e+03,   1.00000000e+00],
       [  0.00000000e+00,   0.00000000e+00,   1.00000000e+00, ...,
          1.00000000e+03,   5.00000000e+03,   1.00000000e+00],
       ..., 
       [  0.00000000e+00,   0.00000000e+00,   1.00000000e+00, ...,
          2.00000000e+03,   3.10000000e+03,   0.00000000e+00],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
          5.29640000e+04,   1.80400000e+03,   0.00000000e+00],
       [  0.00000000e+00,   0.00000000e+00,   1.00000000e+00, ...,
          1.00000000e+03,   1.00000000e+03,   0.00000000e+00]])

In [55]:
train_test=pd.DataFrame(X)
# Split your data
train = train_test[train_test[91] == 1.0]
test = train_test[train_test[91] == 0.0]

# Drop your labels
train = train.drop(91, axis=1)
test = test.drop(91, axis=1)

In [56]:
train=train.values
test=test.values

In [59]:
X=train

In [60]:
#per semplicita dividiamo in train e test anche il nostro train almeno siamo in grado di calcolare qualche metriche per vedere come il nostro algoritmo risponde o meglio "impara"
from sklearn.cross_validation import train_test_split, cross_val_score
from sklearn.feature_selection import SelectKBest, chi2, f_classif

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

**Applichiamo Random Forest**

In [61]:
from sklearn.ensemble import RandomForestClassifier
Rf=RandomForestClassifier()
Rf.fit(X_train, y_train)

# Predicting the Test set results
y_pred = Rf.predict(X_test)

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

[[4368  278]
 [ 944  410]]


0.79633333333333334

In [62]:
from sklearn.ensemble import GradientBoostingClassifier
classifier = GradientBoostingClassifier()
#min_samples_leaf=15
classifier.fit(X_train, y_train)

# Predicting the Test set results
y_pred = classifier.predict(X_test)

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

[[4402  244]
 [ 877  477]]


0.8131666666666667

Try to standardize features to imporove accuracy

In [63]:
from sklearn.preprocessing import StandardScaler
#scale the X coloumns
sc_X = StandardScaler()
#for the training set we need to fit it, then scale it
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

Ripetiamo i nostri algoritmi per vedere se migliorano

In [64]:
from sklearn.ensemble import RandomForestClassifier
Rf=RandomForestClassifier()
Rf.fit(X_train, y_train)

# Predicting the Test set results
y_pred = Rf.predict(X_test)

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

[[4352  294]
 [ 934  420]]


0.79533333333333334

In [65]:
from sklearn.ensemble import GradientBoostingClassifier
classifier = GradientBoostingClassifier()
#min_samples_leaf=15
classifier.fit(X_train, y_train)

# Predicting the Test set results
y_pred = classifier.predict(X_test)

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

[[4401  245]
 [ 877  477]]


0.81299999999999994

La standardizzazione come si puo vedere sembra non essere rilevante ai fini della classificazione.
IL *gradient boosting* comunque risulta migliore del *random forest*:
* Si potrebbe provare un Grid Search su entrambi per ottenere un miglioramento

*GradientBoostingClassifier* **Grid Search**

In [ ]:
#Gradient Boosting grid search
from sklearn.model_selection import GridSearchCV
#specify the parameters for wich we want to find the optimal values, give different options in dictionaries
parameters = [{'loss':["deviance","exponential"],'learning_rate': [0.1, 0.05, 0.2], 'n_estimators': [100,1000,2000], "max_depth":[1,3,5]}]
#estimator: the model
#param_grid: the list of parameters
#scoring: scoring metric
#cv: number of folds in k-fold cross validation
#n_jobs: -1 gives all the power available of the machine
grid_search = GridSearchCV(estimator = GradientBoostingClassifier(),
                           param_grid = parameters,
                           scoring = 'accuracy',
                           n_jobs = -1)
#fit
grid_search = grid_search.fit(X_train, y_train)
#get best accuracy score
best_accuracy = grid_search.best_score_
#best paramesters
best_parameters = grid_search.best_params_

In [103]:
from sklearn.ensemble import GradientBoostingClassifier
classifier = GradientBoostingClassifier(loss="deviance",learning_rate = .5, 
                                  n_estimators = 5000,max_depth=2,min_samples_leaf=15,min_samples_split=40)
#min_samples_leaf=15
classifier.fit(X_train, y_train)

# Predicting the Test set results
y_pred = classifier.predict(X_test)

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

[[4097  549]
 [ 830  524]]


0.77016666666666667

*RandomForestClassifier* **Grid Search**

In [109]:
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]
}
# Create a based model
rf = RandomForestClassifier()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1)
grid_search = grid_search.fit(X_train, y_train)
#get best accuracy score
best_accuracy = grid_search.best_score_
#best paramesters
best_parameters = grid_search.best_params_

KeyboardInterrupt: 

In [110]:
from sklearn.ensemble import RandomForestClassifier
Rf=RandomForestClassifier()
Rf.fit(X_train, y_train)

# Predicting the Test set results
y_pred = Rf.predict(X_test)

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

[[4366  280]
 [ 925  429]]


0.79916666666666669

**Cross Validation**

In [111]:
from sklearn.cross_validation import cross_val_score
RFC = GradientBoostingClassifier(loss="exponential",learning_rate = .2, 
                                  n_estimators = 2000,max_depth=1,min_samples_leaf=15,min_samples_split=40)
y_pred = RFC.fit(X_train, y_train).predict(X_test)
cv_score = cross_val_score(RFC, X_test, y_test, scoring = 'accuracy', cv = 5)
print('CV Model Accuracy :' , cv_score.mean())
print("Number of mislabeled points out of a total %d points : %d" % (X_test.shape[0],(y_test != y_pred).sum()))



CV Model Accuracy : 0.807001890974
Number of mislabeled points out of a total 6000 points : 1125


**Bilanciamento Dataset**

In [66]:
#proviamo a bilanciare il dataset per migliorare un po il risultato
from imblearn.over_sampling import SMOTE
from sklearn.externals.joblib.parallel import _backend

sm = SMOTE(random_state=0, ratio = 1.0)
X, y = sm.fit_sample(X, y)


# Splitting the dataset into the Training set and Test set
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

# Feature Scaling
from sklearn.preprocessing import StandardScaler
#scale the X coloumns
sc_X = StandardScaler()
#for the training set we need to fit it, then scale it
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)


In [67]:
from sklearn.ensemble import RandomForestClassifier
Rf=RandomForestClassifier()
Rf.fit(X_train, y_train)

# Predicting the Test set results
y_pred = Rf.predict(X_test)

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

[[4266  346]
 [ 907 3747]]


0.86477444420461902

In [68]:
from sklearn.ensemble import GradientBoostingClassifier
classifier = GradientBoostingClassifier()
#min_samples_leaf=15
classifier.fit(X_train, y_train)

# Predicting the Test set results
y_pred = classifier.predict(X_test)

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

[[4373  239]
 [ 916 3738]]


0.87535074465788909

Il punteggio dell'accuracy sembra essere di gran lunga migliorato..come in precedenza il gradien boosting sembra essere, anche se di poco, più performante del random forest

### Siamo pronti per fittare il nostro algoritmo sul test reale, al quale manca la variabile target (DEFAULT)

In [69]:
classifier.predict(test)

array([1, 1, 1, ..., 1, 1, 1])